# Sparkify Data Lake

### import important libraries

In [1]:
# import important libraries
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1666765975122_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### set input data path and output data path

In [2]:
# set input data path
input_data = "s3a://udacity-dend/"
# set output data path
output_data = "s3://ahmed-datalake-sparkify/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## process_song_data

> create songs table

> create artists table

> write songs table to parquet files partitioned by year and artist

> write artists table to parquet files

In [3]:
# get filepath to song data file
song_data = os.path.join(input_data, 'song_data/*/*/*/*.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# read song data file
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# extract columns to create songs table
songs_table = df.select(['song_id','title','artist_id','year','duration'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.parquet(os.path.join(output_data, 'songs'), partitionBy=['year', 'artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# extract columns to create artists table
artists_table = df.select(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# write artists table to parquet files
artists_table.write.parquet(os.path.join(output_data, 'artists'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## process_log_data

> extract columns for users table

> write users table to parquet files

> extract columns to create time table

> write time table to parquet files partitioned by year and month

> extract columns from joined song and log datasets to create songplays table 

> write songplays table to parquet files partitioned by year and month

In [9]:
# get filepath to log data file
log_data = os.path.join(input_data, 'log_data/*/*/*.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# read log data file
df1 = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# filter by actions for song plays
df1 = df1.where(df1.page == 'NextSong')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# extract columns for users table 
users_table = df1.select(['userId', 'firstName', 'lastName', 'gender', 'level'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# drop duplicates from use
users_table = users_table.drop_duplicates(subset=['userId'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
users_table.write.parquet(os.path.join(output_data, 'users'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df1 = df1.withColumn('timestamp', get_timestamp('ts'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
time_table =  df1.select(
                F.col("timestamp").alias("start_time"),
                F.hour("timestamp").alias('hour'),
                F.dayofmonth("timestamp").alias('day'),
                F.weekofyear("timestamp").alias('week'),
                F.month("timestamp").alias('month'), 
                F.year("timestamp").alias('year'), 
                F.date_format(F.col("timestamp"), "E").alias("weekday")
            )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
time_table.write.parquet(os.path.join(output_data, 'time'), partitionBy=['year', 'month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
songplays_table = df1.join(
    df,
    (df1.song == df.title) & 
    (df1.artist == df.artist_name) &
    (df1.length == df.duration) &
    (year(df1.timestamp) == df.year),
    'left_outer').select(
    df1.timestamp.alias("start_time"),
    df1.userId.alias("user_id"),
    df1.level,
    df.song_id,
    df.artist_id,
    df1.sessionId.alias("session_id"),
    df1.location,
    df1.userAgent.alias("user_agent"),
    year(df1.timestamp).alias('year'),
    month(df1.timestamp).alias('month')).orderBy("start_time", "user_id").withColumn("songplay_id", F.monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
songplays_table.write.parquet(os.path.join(output_data, 'songplays'), partitionBy=['year', 'month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…